In [119]:
import pandas as pd
import numpy as np
import panel as pn
!pip install hvplot
import hvplot.pandas  # noqa
pn.extension('tabulator')

In [120]:
# df = pd.read_csv('/content/Chicago.csv')

In [121]:
# cache data to improve dashboard performance
if 'data' not in pn.state.cache.keys():

    df = pd.read_csv("Chicago.csv")

    pn.state.cache['data'] = df.copy()

else:

    df = pn.state.cache['data']

In [122]:
df

,Geography Type,Year,Geography,Population - Total,Population - Age 0-17,Genz,Millennial,Population - Age 40-49,Genx,Population - Age 60-69,...,Population - Age 18+,Population - Age 65+,Population - Female,Population - Male,Population - Latinx,Population - Asian Non-Latinx,Population - Black Non-Latinx,Population - White Non-Latinx,Population - Other Race Non-Latinx,Record ID
0,Citywide,2018,Chicago,2705988,548999,552935,456321,336457,312965,262991,...,2156989,349712,1386113,1319875,776661,179841.0,784266.0,899980,119467.0,Citywide-Chicago-2018
1,ZIP Code,2018,60601,14675,820,4606,2792,2190,1333,1340,...,13855,2075,7484,7191,1274,NaN,NaN,9677,NaN,ZIP_Code-60601-2018
2,ZIP Code,2018,60602,1244,149,435,462,135,53,10,...,1095,5,551,693,81,NaN,NaN,788,NaN,ZIP_Code-60602-2018
3,ZIP Code,2018,60603,1174,56,561,101,97,197,97,...,1118,112,601,573,115,NaN,NaN,707,NaN,ZIP_Code-60603-2018
4,ZIP Code,2018,60604,782,38,303,104,51,101,130,...,744,93,413,369,34,NaN,NaN,479,NaN,ZIP_Code-60604-2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,Zip Code,2021,60626,50295,8007,11937,8965,6995,6738,4360,...,42288,5098,25318,24977,9694,2724.0,12646.0,23029,2202.0,ZIP_CODE-60626-2021
231,Zip Code,2021,60628,64489,13734,11638,7491,7593,9066,6930,...,50755,10940,34961,29528,2775,79.0,59168.0,1462,1005.0,ZIP_CODE-60628-2021
232,Zip Code,2021,60629,107487,29283,20690,13227,14954,11096,8187,...,78204,13746,53753,53734,79111,595.0,18878.0,7650,1253.0,ZIP_CODE-60629-2021
233,Zip Code,2021,60630,55591,11093,7053,8926,7337,7479,7975,...,44498,9658,27787,27804,14180,7122.0,1007.0,31325,1957.0,ZIP_CODE-60630-2021


  (0) Minor data processing

In [123]:
df['Geography'] = np.where(df['Geography'] == 'Citywide', '00000', df['Geography'])

In [124]:

# Make DataFrame Pipeline Interactive
df = df.interactive()

 (1) Population overtime by year & age


In [125]:
# Define Panel widgets
year_slider = pn.widgets.IntSlider(name='Year slider', start=2018, end=2021, step=1, value=2020)
year_slider

IntSlider(end=2021, name='Year slider', start=2018, value=2020)

In [126]:
# Radio buttons for Population Gen z Age measure
yaxis_Genz = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['Genz','Genx','Millennial'],
    button_type='success',
  # Default selected value for demonstration
)


In [134]:
 # (2) Table

In [ ]:
Geography = ['60609','60635','60649','60624']
population_total_pipeline = (
    df[
(df['Year'] <= year_slider) &
        (df['Geography'].isin(Geography))  # Filter by Geography list
    ].groupby(['Geography', 'Year','Millennial','Genx'])[yaxis_population_total.value].mean()  # Group by 'Geography' and 'Year'
    .reset_index()  # Reset the index to include 'Geography' and 'Year' as columns
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)
population_total_pipeline





(3) Table - Population over time by year

In [ ]:
Population_table = population_total_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width')
Population_table

(4) Population Age Scatterplot

In [ ]:
Population_plot = population_total_pipeline.hvplot.scatter(
    x='Year',
    by='Geography',
    y=yaxis_Genz,  # Assuming 'yaxis_population_age' is defined
    line_width=2,
    title=" Gen Z Population by Zipcode"
)
Population_plot


(5) Barchart with Population Total Sources by Zip Code


In [ ]:
yaxis_population_total_source = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['Genx', 'Millennial','Genz'],
    button_type='success'
)

Geography_excl_world = ['60609', '60624', '60649','60635']

population_total_source_bar_pipeline = (
    df[
        (df['Year'] >= year_slider.value) &
        (df['Geography'].isin(Geography))  # Filter by Geography list
    ].groupby(['Geography','Genx','Millennial','Year','Genz' ])['Population - Total'].mean()  # Group by 'Geography' and 'Population_Age18-29', aggregate 'Population - Total'
    .reset_index()  # Reset the index
    .reset_index()
    .sort_values(by='Population - Total')
    .reset_index(drop=True)
)

In [ ]:
population_total_source_bar_plot = population_total_source_bar_pipeline.hvplot(kind='bar',
                                                     x='Geography',
                                                     y=yaxis_population_total_source,
                                                     title='Population Age by Zipcode ')
population_total_source_bar_plot

In [132]:
# (6) Population by Gender

In [ ]:
# Radio buttons for gender measures
yaxis_gender = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Population - Male', 'Population - Female', 'Population - Total'],
    button_type='success'
)
Year_excl_world = ['2018', '2019', '2020', '2021']


gender_pipeline = (
     df[
        (df['Year'] <= year_slider.value) &
        (df['Geography'].isin(Geography))  # Filter by Geography list
    ].groupby(['Year','Geography', 'Population - Male', 'Population - Female'])['Population - Total'].mean()  # Group by 'Geography' and 'Population_Age18-29', aggregate 'Population - Total'
    .reset_index()  # Reset the index
    .reset_index()
    .sort_values(by='Year')
    .reset_index(drop=True)
)

gender_pipeline
gender_plot = gender_pipeline.hvplot(x = 'Year', by = 'Geography', y=yaxis_gender,line_width=2, title="Gender by Population")
gender_plot

(7) Creating Dashboard

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Chicago Population dashboard',
    sidebar=[pn.pane.Markdown("# Chicago Facts"),
             pn.pane.Markdown("#### Chicago is the third largest city in the united states with over 3 million people living there. As of 2020, Chicago's population was 2,746,388. The city's population peaked in 1950 at 3,621,092, but has since declined.For every 100 females in Chicago, there are 94.2 males.Chicago's population is also broken down by age group: 26.2% are under 18, 11.2% are 18–24, 33.4% are 25–44, 18.9% are 45–64, and 10.3% are 65 and older. The median age is 32."),
             pn.pane.PNG('Chicago Skyline.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"), year_slider],
        main=[pn.Row(pn.Column( yaxis_Genz,
                           Population_plot.panel(width=700), margin=(0,25)),
                 Population_table.panel(width=500)),
          pn.Row(pn.Column(yaxis_gender, gender_plot.panel(width=600)),
              pn.Column(yaxis_population_total_source, population_total_source_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.show();